**ETF TAX Calculator AUSTRIA**

Data provided by https://my.oekb.at/kapitalmarkt-services/kms-output/fonds-info/sd/af/f?isin=IE00BFY0GT14

Calculation is made in the following way :
$$
\begin{align*}
\text{Capital\_Gain} &= \text{Österreichische\_KESt} \times \text{\#Shares} \\
\text{Taxable\_ETF\_Gain} &= \text{Value\_Year\_After} - \text{Value\_Year\_Before}  \\
\text{Tax\_Paid} &=  \frac {\text{Capital\_Gain}}{\text{USDEUR}} \\
\text{Percentage\_Tax\_Paid} &= \frac {\text{Österreichische KESt} \times 100}{\text{Taxable\_ETF\_Gain} \times \text{USDEUR}} \\
\text{ETF\_New\_Average\_Cost} &= \text{ETF\_Initial\_Cost} + ({\text{Fondsergebnis der Meldeperiode}} \times {\text{USDEUR}})
\end{align*}
$$

Where:


* {Österreichische KESt} is the value provided by the Fund to Öekb (USD).
* Value_Year_After  (USD) is the ETF value at the date when KESt is provided.
* Value_Year_Before (USD) is the ETF value one year before.
* Fondsergebnis der Meldeperiode (USD) is the value to add to the original ETF Cost (when you bought the shares)
* USDEUR is computed at the date when KESt is provided.

In [1]:
from modules import general_utils
from modules import stock_utils
import pandas as pd
from pandas import DataFrame
import warnings
import time
import os

**WAY TO AUTOMATIZE**


In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains # Importa ActionChains
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException

def get_text_from_element(tag, selectors, max_retries=5):
    service = Service(executable_path=chromedriver_path)
    options = Options()
    options.binary_location = chrome_path
    retries = 0
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)

    while retries < max_retries:
            try:
                # Wait for the page to be fully loaded
                WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located((By.TAG_NAME, tag))
                )

                for selector in selectors:
                    try:
                        # Use CSS selector
                        element = WebDriverWait(driver, 30).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, selector))
                        )

                        # Ensure the element is clickable (visible and enabled)
                        WebDriverWait(driver, 30).until(
                            EC.visibility_of(element)
                        )

                        # Extract the text from the element
                        text = element.text
                        print(f"Extracted Text using selector '{selector}': {text}")

                    except (TimeoutException, StaleElementReferenceException):
                        # If the selector fails, try the next one
                        print(f"Selector '{selector}' failed. Trying next selector...")
                        continue
                    
                # If all selectors fail, raise an exception
                raise Exception("All selectors failed to locate the element.")

            except StaleElementReferenceException:
                print("Stale element reference detected. Retrying...")
                retries += 1
                if retries == max_retries:
                    raise Exception("Max retries reached. Could not locate the element.")
    

def insert_isin_and_get_results(url, chrome_path, chromedriver_path, isin):
    """Inserisce ISIN, gestisce i cookie, clicca il pulsante e restituisce i risultati della tabella."""

    try:
        
        service = Service(executable_path=chromedriver_path)

        options = Options()
        options.binary_location = chrome_path
        # options.add_argument('--headless=new')
        
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        # Gestione del banner dei cookie (con XPath corretto e scroll)
        try:
            # Attesa esplicita per il banner
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//*[@id='cookie-modal']/div/div")))

            cookie_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='cookie-modal']/div/div/div[1]/div[4]/div[2]/p-button/button/span")))
            actions = ActionChains(driver)
            actions.move_to_element(cookie_button).perform()
            cookie_button.click()

            WebDriverWait(driver, 30).until(EC.invisibility_of_element_located((By.XPATH, "//*[@id='cookie-modal']/div/div"))) #attesa che sparisca il div
        except (TimeoutException, NoSuchElementException) as e:
            print(f"Errore gestione Cookie: {e}")
            driver.save_screenshot("cookie_error.png")
    
        # Inserisci l'ISIN
        isin_input = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='customer-portal']/div[1]/div/div/app-kms-output-root/lib-kupl-kms-fond-info/lib-bezahl-data/kupl-page-template-sidebar/div/sidebar/div/div[2]/div[2]/div/kupl-autocomplete-filter/div/div/p-autocomplete/div/input"))
        )
        isin_input.clear()
        isin_input.send_keys(isin)
        
        isin_to_click = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='pn_id_12_0']/div/b"))
            )
        isin_to_click.click()

        # CSS_SELECTOR is reliable when XPATH has Dynamic ID inside
        
        button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "tr.master span"))
            )
        button.click()

        # Define the selectors from your JSON data
        selectors = [
        "table > tbody > tr.p-element.master.ng-star-inserted.p-highlight-contextmenu > td:nth-child(5) > div"
        ]

        try:
            text = get_text_from_element("body",selectors)
        finally:
            driver.quit()



    except TimeoutException as e:

        
        print(f"Timeout: {e}")
        #if 'driver' in locals():
            #driver.save_screenshot("timeout.png")
    except Exception as e:
        print(f"Errore generico: {e}")
        #if 'driver' in locals():
            #driver.save_screenshot("errore_generico.png")
    finally:
        if 'driver' in locals():
            driver.quit()


chrome_path = r"C:\Users\david\Downloads\chrome-win64\chrome-win64\chrome.exe"
chromedriver_path = r"C:\Users\david\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
isin_to_insert = "IE00BFY0GT14"
#url = "https://www.pfennigfuchser.at/der-etf-rendite-rechner/"
url = "https://my.oekb.at/kapitalmarkt-services/kms-output/fonds-info/sd/af/f"
insert_isin_and_get_results(url, chrome_path, chromedriver_path, isin_to_insert)


Errore generico: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.264)
Stacktrace:
	GetHandleVerifier [0x00007FF652F380D5+2992373]
	(No symbol) [0x00007FF652BCBFD0]
	(No symbol) [0x00007FF652A6590A]
	(No symbol) [0x00007FF652A3F4F5]
	(No symbol) [0x00007FF652AE63A7]
	(No symbol) [0x00007FF652AFEE72]
	(No symbol) [0x00007FF652ADF113]
	(No symbol) [0x00007FF652AAA918]
	(No symbol) [0x00007FF652AABA81]
	GetHandleVerifier [0x00007FF652F96A2D+3379789]
	GetHandleVerifier [0x00007FF652FAC32D+3468109]
	GetHandleVerifier [0x00007FF652FA0043+3418211]
	GetHandleVerifier [0x00007FF652D2C78B+847787]
	(No symbol) [0x00007FF652BD757F]
	(No symbol) [0x00007FF652BD2FC4]
	(No symbol) [0x00007FF652BD315D]
	(No symbol) [0x00007FF652BC2979]
	BaseThreadInitThunk [0x00007FFC80DAE8D7+23]
	RtlUserThreadStart [0x00007FFC824DBF2C+44]



KeyboardInterrupt: 

: 